In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
DATASET_LOCATION = '/content/drive/MyDrive/ai-playground/ra-text-classification-spectrogram/spectrograms/bert-isot4000'
data_dir = pathlib.Path(DATASET_LOCATION)

In [4]:
TRAIN_FRACTION = 0.8
batch_size = 32 # default batch size of data
img_height = 224 # default height to which image is resized
img_width = 224 # default width to which image is resized
seed = 2022 # random seed for shuffling and transformations

In [5]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "training",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 3032 files belonging to 2 classes.
Using 2426 files for training.


In [6]:
ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "validation",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 3032 files belonging to 2 classes.
Using 606 files for validation.


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58889256/58889256 [==============================] - 0s 0us/step


In [9]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [10]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [11]:
model = tf.keras.Model(pre_trained_model.input, x)

In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
76/76 [==============================] - 959s 3s/step - loss: 6.5155 - accuracy: 0.5763 - val_loss: 0.6973 - val_accuracy: 0.7970
Epoch 2/100
76/76 [==============================] - 14s 188ms/step - loss: 1.1171 - accuracy: 0.6719 - val_loss: 0.5894 - val_accuracy: 0.6749
Epoch 3/100
76/76 [==============================] - 14s 187ms/step - loss: 0.5951 - accuracy: 0.7214 - val_loss: 0.4489 - val_accuracy: 0.7987
Epoch 4/100
76/76 [==============================] - 14s 187ms/step - loss: 0.5298 - accuracy: 0.7564 - val_loss: 0.4460 - val_accuracy: 0.7987
Epoch 5/100
76/76 [==============================] - 14s 186ms/step - loss: 0.5543 - accuracy: 0.7193 - val_loss: 0.6107 - val_accuracy: 0.6551
Epoch 6/100
76/76 [==============================] - 14s 186ms/step - loss: 0.5143 - accuracy: 0.7560 - val_loss: 0.4657 - val_accuracy: 0.7921
Epoch 7/100
76/76 [==============================] - 14s 187ms/step - loss: 0.5137 - accuracy: 0.7531 - val_loss: 0.4605 - val_accuracy: 0